### Commercial Implications
A Deep Research agent is broadly applicable to any business area, and to your own day-to-day activities. You can make use of this yourself!

In [1]:

from agents import Agent, WebSearchTool, trace, Runner, gen_trace_id, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import asyncio
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from typing import Dict
from IPython.display import display, Markdown

In [2]:

load_dotenv(override=True)

True

### **OpenAI Hosted Tools**

OpenAI Agents SDK includes the following hosted tools:

The `WebSearchTool` lets an agent search the web.  
The `FileSearchTool` allows retrieving information from your OpenAI Vector Stores.  
The `ComputerTool` allows automating computer use tasks like taking screenshots and clicking.  

#### *Important note - API charge of WebSearchTool*

This is costing me 2.5 cents per call for OpenAI WebSearchTool. That can add up to 
3 for the next 2 labs. We'll use low cost Search tools with other platforms, so feel free to skip running this if the cost is a concern.

In [8]:
INSTRUCTIONS = """
    You are a research assistant. Given a search term, you search the web for
    that term and produce a concise summary of the results. The summary must be 
    2-3 paragraphs and less than 300 words. Capture the main points. Write succintly,
    no need to have the complete sentence or good grammar. This will be consumed nt someone 
    synthesizing a report, so it's vital you capture the essence and ignore any fluff.
    Do not include any addtional commentary other than the summary itself.
"""


search_agent = Agent(
    name="Search Agent",
    instructions=INSTRUCTIONS,
    model = 'gpt-4o-mini',
    tools=[WebSearchTool(search_context_size="low")],
    model_settings=ModelSettings(tool_choice="required")
)

In [9]:
message = "Latest AI Agent Frameworks in 2025."

with trace("Search"):
    result = await Runner.run(search_agent, message)

display(Markdown(result.final_output))

As of mid-2025, several AI agent frameworks have emerged, each offering unique capabilities for developing intelligent, autonomous systems.

**LangChain** is a versatile framework that facilitates the creation of AI applications by integrating various tools, models, and data sources. Its modular design allows developers to build complex workflows efficiently. ([dev.to](https://dev.to/surgedatalab/best-5-frameworks-for-agentic-ai-in-2025-enabling-next-gen-intelligent-multi-agent-systems-40ce?utm_source=openai))

**LangGraph**, an extension of LangChain, introduces a graph-based system for managing agent workflows. This approach is particularly useful for applications requiring complex decision-making processes, such as loan processing or insurance claims. ([phyniks.com](https://phyniks.com/blog/top-7-agentic-ai-frameworks-in-2025?utm_source=openai))

**AutoGen**, developed by Microsoft, automates the generation of code and models needed for complex workflows. It leverages large language models to assist developers in building, fine-tuning, and deploying AI solutions with minimal manual coding, making it accessible even to those without extensive AI expertise. ([blog.mechcloud.io](https://blog.mechcloud.io/top-5-ai-agent-frameworks-in-2025?utm_source=openai))

**CrewAI** is designed for orchestrating collaborative AI agents in complex, multi-step workflows. Its role-based architecture enables developers to define agents with specialized responsibilities, facilitating the creation of systems that mimic human decision-making processes. ([dev.to](https://dev.to/surgedatalab/best-5-frameworks-for-agentic-ai-in-2025-enabling-next-gen-intelligent-multi-agent-systems-40ce?utm_source=openai))

**Eliza** is an open-source, Web3-friendly AI agent operating system that seamlessly integrates with blockchain applications. It allows developers to create AI agents capable of interacting with smart contracts and blockchain data, expanding the potential applications of AI in decentralized environments. ([arxiv.org](https://arxiv.org/abs/2501.06781?utm_source=openai))

These frameworks represent the forefront of AI agent development in 2025, each contributing to the advancement of intelligent, autonomous systems across various domains. 

### **We will now use Structured Outputs, and include a description of the fields**

In [12]:
HOW_MANY_SEARCHES = 3

INSTRUCTIONS = f"You are a helpful research assistant. Given a query, come up with \
a set of web searches to perform to best answer the query. Output {HOW_MANY_SEARCHES} terms to query for"


class WebSearchItem(BaseModel):
    reason: str
    """Your reasoning for why this search is important for this query."""

    query: str
    """The search term to use for the web search."""


class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem]
    """A list of web searches to perform to best answer the query."""



planner_agent = Agent(
    name="Planner Agent",
    instructions=INSTRUCTIONS, 
    model = 'gpt-4o-mini',
    output_type = WebSearchPlan
)

In [13]:
message = "Latest AI Agent Frameworks in 2025."

with trace("Search"):
    result = await Runner.run(planner_agent, message)
    print(result.final_output)

searches=[WebSearchItem(reason='To find recent developments and releases in AI agent frameworks specifically in the year 2025.', query='latest AI agent frameworks 2025'), WebSearchItem(reason='To gather information on the most popular and widely used AI frameworks and platforms in 2025.', query='popular AI frameworks 2025'), WebSearchItem(reason='To explore academic research and industry reports on advancements in AI frameworks for agents in 2025.', query='AI agent frameworks research 2025')]


In [14]:
@function_tool
def send_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("jai.300012723027@csvtu.ac.in") # Change this to your verified email
    to_email = To("ksharma9719@gmail.com") # Change this to your email
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [15]:
send_email

FunctionTool(name='send_email', description='Send out an email with the given subject and HTML body', params_json_schema={'properties': {'subject': {'title': 'Subject', 'type': 'string'}, 'html_body': {'title': 'Html Body', 'type': 'string'}}, 'required': ['subject', 'html_body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001DA08A3F7E0>, strict_json_schema=True, is_enabled=True)

In [16]:
INSTRUCTIONS = """
    You are able to send a nicely formatted HTML email based on a detailed report.
    You will be provided with detailed report. You should use your tool to send one email, 
    providing the report converted into clean, well presented HTML with an appropriate subject line.
"""

email_agent = Agent(
    name = "Email Agent",
    instructions = INSTRUCTIONS,
    model='gpt-4o-mini',
    tools=[send_email]
)

In [17]:
# report writer agent


INSTRUCTIONS = (
    "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "You will be provided with the original query, and some initial research done by a research assistant.\n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 5-10 pages of content, at least 1000 words."
)


class ReportData(BaseModel):
    short_summary: str = Field(description="A short 2-3 sentence summary of the findings.")

    markdown_report: str = Field(description="The final report.")

    follow_up_questions: list[str] = Field(description="Suggested topics to research further.")



writer_agent = Agent(
    name ="Writer Agent",
    instructions = INSTRUCTIONS,
    model = 'gpt-4o-mini',
    output_type = ReportData
)

> 
The next 3 functions will plan and execute the search, using planner_agent and search_agent

In [18]:
async def plan_searches(query: str):
    """ Use the planner_agent to plan which searches to run for the query """
    print("Planning searches...")
    result = await Runner.run(planner_agent, f"Query: {query}")
    print(f"Will perform {len(result.final_output.searches)} searches")
    return result.final_output


async def perform_searches(search_plan: WebSearchPlan):
    """ Call search() for each item in the search plan """
    print("Searching...")
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    print("Finished searching")
    return results


async def search(item: WebSearchItem):
    """ Use the search agent to run a web search for each item in the search plan """
    input = f"Search term: {item.query}\nReason for searching: {item.reason}"
    result = await Runner.run(search_agent, input)
    return result.final_output

In [19]:
async def write_report(query: str, search_results: list[str]):
    """ Use the writer agent to write a report based on the search results"""
    print("Thinking about report...")
    input = f"Original query: {query}\nSummarized search results: {search_results}"
    result = await Runner.run(writer_agent, input)
    print("Finished writing report")
    return result.final_output


async def send_email(report: ReportData):
    """ Use the email agent to send an email with the report """
    print("Writing email...")
    result = await Runner.run(email_agent, report.markdown_report)
    print("Email sent")
    return report

### **Showtime!**

In [20]:
query ="Latest AI Agent frameworks in 2025"

with trace("Research trace"):
    print("Starting research...")
    search_plan = await plan_searches(query)
    search_results = await perform_searches(search_plan)
    report = await write_report(query, search_results)
    await send_email(report)
    print("Email sent!")

Starting research...
Planning searches...
Will perform 3 searches
Searching...
Finished searching
Thinking about report...
Finished writing report
Writing email...
Email sent
Email sent!


#### Always look at the traces: https://platform.openai.com/traces
